In [2]:
# -------------------------
# Import necessary libraries
# -------------------------
import heapq  # Priority queue for event scheduling
import math
import pandas as pd
import numpy as np
import random

# -------------------------
# Simulation configuration constants
# -------------------------
SIMULATION_START = 0       # Start time of the simulation
SIMULATION_END = 600       # Last arrival time allowed
CLOSE_TIME = 660           # Time at which workers stop processing new tasks

# -------------------------
# Linear Congruential Generator (LCG) for pseudo-random numbers
# -------------------------
class LCG:
    def __init__(self, seed=1):
        self.a = 1664525
        self.c = 1013904223
        self.m = 2 ** 32
        self.state = seed

    def random(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / self.m  # Return a float in [0, 1)

# -------------------------
# Random variable generators using LCG
# -------------------------
def generate_normal_direct(mu, sigma_squared, lcg=None):
    # Direct method for generating normal distribution using Box-Muller transform
    rand = lcg.random if lcg else random.random
    R1 = rand()
    R2 = rand()
    Z = math.sqrt(-2 * math.log(R1)) * math.cos(2 * math.pi * R2)
    return max(0.1, mu + math.sqrt(sigma_squared) * Z)  # Ensure positive durations

# Customer arrival time
def next_arrival(lcg):
    return generate_normal_direct(30, 900, lcg)

# Rental duration for K or M
def rental_time(person, lcg):
    return generate_normal_direct(14, 16, lcg) if person == "K" else generate_normal_direct(10, 25, lcg)

# Tool usage duration
def usage_duration(lcg):
    return generate_normal_direct(60, 3600, lcg)

# Maintenance and cleaning times
def maintenance_time(lcg):
    return generate_normal_direct(6, 16, lcg)

def cleaning_time(lcg):
    return generate_normal_direct(10, 36, lcg)

# -------------------------
# Event class for simulation scheduling
# -------------------------
class Event:
    def __init__(self, time, type, data=None):
        self.time = time    # Timestamp of the event
        self.type = type    # Event type (e.g., "arrival", "end_rent")
        self.data = data    # Optional data (e.g., customer ID or worker ID)

    def __lt__(self, other):
        return self.time < other.time  # Enables min-heap behavior

# -------------------------
# Reset all simulation state
# -------------------------
def reset_state():
    return {
        "fel": [],               # Future event list (min-heap)
        "clock": SIMULATION_START,
        "queue": [],             # Waiting customers
        "return_queue": [],      # Customers who returned tools
        "to_maintain": [],       # Tools awaiting maintenance
        "to_clean": [],          # Tools awaiting cleaning
        "K_busy": False,
        "M_busy": False,
        "K_status": "idle",
        "M_status": "idle",
        "active_customers": set(),
        "stats": {
            "wait_times": [],
            "queue_lengths": [],
            "rent_times_K": [],
            "rent_times_M": [],
            "maint_times": [],
            "clean_times": [],
            "timeline": [],
            "wait_over_5min_count": 0,
            "total_customers_with_wait": 0,
            "count_rent_K": 0,
            "count_rent_M": 0,
            "count_maint": 0,
            "count_clean": 0
        },
        "customer_count": 0,
    }

# -------------------------
# Event Handlers
# -------------------------
def handle_arrival(state, lcg):
    # Handle arrival of a new customer
    now = state["clock"]
    cid = f"W{state['customer_count']}"
    state["customer_count"] += 1
    state["queue"].append((now, cid))
    state["active_customers"].add(cid)
    state["stats"]["timeline"].append((now, "arrival", cid))

    # Schedule return event if within closing time
    return_time = now + usage_duration(lcg)
    if return_time < CLOSE_TIME:
        heapq.heappush(state["fel"], Event(return_time, "request_return", cid))

    assign_workers(state, lcg)

    # Schedule next arrival
    next_time = now + next_arrival(lcg)
    if next_time < SIMULATION_END:
        heapq.heappush(state["fel"], Event(next_time, "arrival"))

def handle_end_rent(state, who, lcg):
    # Worker finishes rental task
    state[f"{who}_busy"] = False
    state[f"{who}_status"] = "idle"
    state["stats"]["timeline"].append((state["clock"], "end_rent", who))
    assign_workers(state, lcg)

def handle_end_return(state, customer_id, lcg):
    # Process tool return and add to maintenance queue
    if customer_id in state["active_customers"]:
        state["active_customers"].remove(customer_id)
    state["to_maintain"].append(f"tool_from_{customer_id}")
    state["stats"]["timeline"].append((state["clock"], "returned", customer_id))
    state["stats"]["timeline"].append((state["clock"], "end_return", customer_id))
    for who in ["K", "M"]:
        if state[f"{who}_status"] == "returning":
            state[f"{who}_busy"] = False
            state[f"{who}_status"] = "idle"
    assign_workers(state, lcg)

def handle_end_maint(state, lcg):
    # Maintenance completed, schedule cleaning
    t_clean = cleaning_time(lcg)
    t_maint = maintenance_time(lcg)
    state["K_status"] = "cleaning"
    state["stats"]["count_maint"] += 1
    state["stats"]["maint_times"].append(t_maint)
    state["stats"]["timeline"].append((state["clock"], "end_maint", "K"))
    heapq.heappush(state["fel"], Event(state["clock"] + t_clean, "end_clean"))
    assign_workers(state, lcg)

def handle_end_clean(state, lcg):
    # Cleaning completed
    state["K_status"] = "idle"
    state["K_busy"] = False
    t_clean = cleaning_time(lcg)
    state["stats"]["count_clean"] += 1
    state["stats"]["clean_times"].append(t_clean)
    state["stats"]["timeline"].append((state["clock"], "end_clean", "K"))
    assign_workers(state, lcg)

# -------------------------
# Worker assignment logic
# -------------------------
def assign_workers(state, lcg):
    now = state["clock"]
    if now >= CLOSE_TIME:
        return

    # Priority: handle returns
    for who in ["K", "M"]:
        if not state[f"{who}_busy"] and state["return_queue"]:
            cid = state["return_queue"].pop(0)
            state[f"{who}_busy"] = True
            state[f"{who}_status"] = "returning"
            heapq.heappush(state["fel"], Event(now + 2, "return", cid))
            return

    # Then assign rentals
    for who in ["M", "K"]:
        if not state[f"{who}_busy"]:
            for i, (t0, cid) in enumerate(state["queue"]):
                if cid not in state["active_customers"]:
                    continue
                wait = now - t0
                rent = rental_time(who, lcg)
                state[f"{who}_busy"] = True
                state[f"{who}_status"] = "renting"
                state["stats"]["wait_times"].append(wait)
                state["stats"][f"rent_times_{who}"].append(rent)
                state["stats"]["total_customers_with_wait"] += 1
                state["stats"][f"count_rent_{who}"] += 1
                if wait > 5:
                    state["stats"]["wait_over_5min_count"] += 1
                state["stats"]["timeline"].append((now, "start_rent", f"{who}->{cid}"))
                del state["queue"][i]
                heapq.heappush(state["fel"], Event(now + rent, "end_rent", who))
                return

    # Then assign maintenance
    if not state["K_busy"] and state["to_maintain"]:
        t = maintenance_time(lcg)
        if now + t <= CLOSE_TIME:
            state["to_maintain"].pop(0)
            state["K_busy"] = True
            state["K_status"] = "maintaining"
            state["stats"]["maint_times"].append(t)
            state["stats"]["timeline"].append((now, "start_maint", "K"))
            heapq.heappush(state["fel"], Event(now + t, "end_maint"))
            return

    # Record queue length when no assignment occurs
    state["stats"]["queue_lengths"].append(len(state["queue"]))

# -------------------------
# Execute a single simulation run
# -------------------------
def run_simulation(seed):
    lcg = LCG(seed)
    state = reset_state()
    heapq.heappush(state["fel"], Event(SIMULATION_START, "arrival"))

    while state["fel"]:
        event = heapq.heappop(state["fel"])
        state["clock"] = event.time

        # Dispatch based on event type
        if event.type == "arrival":
            handle_arrival(state, lcg)
        elif event.type == "end_rent":
            handle_end_rent(state, event.data, lcg)
        elif event.type == "return":
            handle_end_return(state, event.data, lcg)
        elif event.type == "end_maint":
            handle_end_maint(state, lcg)
        elif event.type == "end_clean":
            handle_end_clean(state, lcg)
        elif event.type == "request_return":
            state["return_queue"].append(event.data)
            assign_workers(state, lcg)

    return state["stats"]

# -------------------------
# Run the simulation across multiple seeds
# -------------------------
def run_full_simulation(seeds):
    results = []
    for seed in seeds:
        stats = run_simulation(seed)
        results.append({
            "avg_wait": np.mean(stats["wait_times"]) if stats["wait_times"] else 0,
            "avg_rent_K": np.mean(stats["rent_times_K"]) if stats["rent_times_K"] else 0,
            "avg_rent_M": np.mean(stats["rent_times_M"]) if stats["rent_times_M"] else 0,
            "avg_maint": np.mean(stats["maint_times"]) if stats["maint_times"] else 0,
            "avg_clean": np.mean(stats["clean_times"]) if stats["clean_times"] else 0,
            "avg_queue_len": np.mean(stats["queue_lengths"]) if stats["queue_lengths"] else 0,
            "wait_over_5_pct": (stats["wait_over_5min_count"] / stats["total_customers_with_wait"]) * 100 if stats["total_customers_with_wait"] > 0 else 0,
            "count_waits": stats["total_customers_with_wait"],
            "count_rent_K": stats["count_rent_K"],
            "count_rent_M": stats["count_rent_M"],
            "count_maint": stats["count_maint"],
            "count_clean": stats["count_clean"]
        })
    return results

# -------------------------
# MAIN EXECUTION BLOCK
# -------------------------
if __name__ == "__main__":
    # Generate seed values and run simulation
    SEEDS = [i * 17 + 123 for i in range(30)]
    results = run_full_simulation(SEEDS)

    # Create DataFrame from simulation results
    df = pd.DataFrame(results)
    df.index = [f"Run {i+1}" for i in range(len(df))]

    # Compute averages across all runs
    summary_row = pd.Series({
        "avg_wait": df["avg_wait"].mean(),
        "avg_rent_K": df["avg_rent_K"].mean(),
        "avg_rent_M": df["avg_rent_M"].mean(),
        "avg_maint": df["avg_maint"].mean(),
        "avg_clean": df["avg_clean"].mean(),
        "avg_queue_len": df["avg_queue_len"].mean(),
        "wait_over_5_pct": df["wait_over_5_pct"].mean(),
        "count_waits": df["count_waits"].sum(),
        "count_rent_K": df["count_rent_K"].sum(),
        "count_rent_M": df["count_rent_M"].sum(),
        "count_maint": df["count_maint"].sum(),
        "count_clean": df["count_clean"].sum()
    }, name="Average Across Runs")

    df_final = pd.concat([df, pd.DataFrame([summary_row])])

    # Rename columns for readability
    df_final.rename(columns={
        "avg_wait": "Avg Wait (min)",
        "avg_rent_K": "Rental K (min)",
        "avg_rent_M": "Rental M (min)",
        "avg_maint": "Maint. (min)",
        "avg_clean": "Clean. (min)",
        "avg_queue_len": "Avg Queue Length",
        "wait_over_5_pct": "Wait >5 min (%)",
        "count_waits": "Count Waits",
        "count_rent_K": "Count Rent K",
        "count_rent_M": "Count Rent M",
        "count_maint": "Count Maint.",
        "count_clean": "Count Clean."
    }, inplace=True)

    # Add missing scenario columns
    for col in [
        "Rental R (min)", "Count Rent R",
        "Count Maint K", "Count Maint R",
        "Count Clean K", "Count Clean R"
    ]:
        if col not in df_final.columns:
            df_final[col] = 0

    # Split total counts into worker-specific columns
    df_final["Count Maint K"] = df_final["Count Maint."]
    df_final["Count Clean K"] = df_final["Count Clean."]
    df_final["Count Maint M"] = 0
    df_final["Count Clean M"] = 0
    df_final.drop(columns=["Count Maint.", "Count Clean."], inplace=True)

    # Reorder columns for clarity
    ordered_columns = [
        "Avg Wait (min)", "Rental K (min)", "Rental M (min)", "Rental R (min)",
        "Maint. (min)", "Clean. (min)", "Avg Queue Length", "Wait >5 min (%)",
        "Count Waits", "Count Rent K", "Count Rent M", "Count Rent R",
        "Count Maint M", "Count Maint K", "Count Maint R",
        "Count Clean M", "Count Clean K", "Count Clean R"
    ]
    df_final = df_final[ordered_columns]

    # Display configuration settings
    pd.set_option("display.max_rows", None)
    pd.set_option("display.float_format", lambda x: f"{x:.2f}")
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", None)
    pd.set_option("display.expand_frame_repr", False)


    # df_final.to_csv("results_part1.csv", index=False)
    # print("\n📊 Full Results Table (All 30 Runs + Summary):\n")
    # print(df_final)



# -------------------------
# Confidence Interval Calculation
# -------------------------
from scipy.stats import t

# Manual calculation of 95% confidence intervals
def manual_confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / (n - 1)
    std_dev = math.sqrt(variance)
    se = std_dev / math.sqrt(n)
    t_critical = t.ppf((1 + confidence) / 2, df=n - 1)
    lower = mean - t_critical * se
    upper = mean + t_critical * se
    return mean, max(0, lower), upper  # Ensure lower bound is not negative

# Compute confidence intervals for key metrics
metrics = {
    "Average Wait Time (min)": df["avg_wait"],
    "Rental Time - K (min)": df["avg_rent_K"],
    "Rental Time - M (min)": df["avg_rent_M"],
    "Maintenance Time (min)": df["avg_maint"],
    "Cleaning Time (min)": df["avg_clean"],
    "Average Queue Length": df["avg_queue_len"],
    "Wait >5 Minutes (%)": df["wait_over_5_pct"]
}

# Build the CI summary table
ci_results = []
for name, series in metrics.items():
    mean, lower, upper = manual_confidence_interval(series)
    ci_results.append({
        "Metric": name,
        "Point Estimate": round(mean, 2),
        "95% CI Lower": round(lower, 2),
        "95% CI Upper": round(upper, 2)
    })

ci_results_df = pd.DataFrame(ci_results)

# Display confidence interval results
print("\n📌 Confidence Interval Summary Table:\n")
print(ci_results_df.to_string(index=False))



📌 Confidence Interval Summary Table:

                 Metric  Point Estimate  95% CI Lower  95% CI Upper
Average Wait Time (min)            1.01          0.68          1.34
  Rental Time - K (min)           12.76         10.85         14.67
  Rental Time - M (min)           10.09          9.65         10.53
 Maintenance Time (min)            6.08          5.87          6.30
    Cleaning Time (min)            9.99          9.41         10.57
   Average Queue Length            0.39          0.23          0.56
    Wait >5 Minutes (%)            7.33          4.81          9.84
